# Métrica 3_4_02 — Bienes inmuebles adquiridos por el declarante por compraventa
| Escenario                                                                 | Condición                     | Resultado            |
| :------------------------------------------------------------------------ | :---------------------------- | :------------------- |
| No hay bienes inmuebles                                                   | —                             | ⚪ **SIN_DATO**       |
| Bien inmueble no pertenece al declarante                                  | `titular ≠ “DECLARANTE”`      | ⚪ **N/A**            |
| Número de bienes del declarante por compraventa dentro del rango esperado | —                             | 🟢 **CUMPLE (0)**    |
| Número de bienes del declarante por compraventa fuera del rango esperado  | `< Q1` o `> Q3` o umbral fijo | 🔴 **NO_CUMPLE (1)** |


In [1]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "3_4_02_BIENES_COMPRAVENTA_DECLARANTE"
BATCH_SIZE = 10000  # 🔧 Lote de 10 mil documentos

# Rangos estimados por nivel jerárquico (modificables según análisis SESNA)
UMBRAL_POR_NIVEL = {
    "OPERATIVO": (0, 2),
    "ENLACE": (0, 3),
    "JEFE_DEPARTAMENTO": (0, 4),
    "DIRECTOR": (0, 5),
    "SUBDIRECTOR": (0, 5),
    "TITULAR": (0, 6),
}

def extraer_lista(doc, path):
    try:
        partes = path.split(".")
        actuales = [doc]
        for p in partes:
            siguientes = []
            for actual in actuales:
                if isinstance(actual, dict):
                    valor = actual.get(p)
                    if valor is None:
                        continue
                    if isinstance(valor, list):
                        siguientes.extend(valor)
                    else:
                        siguientes.append(valor)
                elif isinstance(actual, list):
                    for sub in actual:
                        if isinstance(sub, dict):
                            valor = sub.get(p)
                            if valor is None:
                                continue
                            if isinstance(valor, list):
                                siguientes.extend(valor)
                            else:
                                siguientes.append(valor)
            actuales = siguientes
        return [a for a in actuales if isinstance(a, dict)]
    except:
        return []

def extraer_valor(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None
        return actual
    except:
        return None

def evaluar_metrica(doc):
    bienes = extraer_lista(doc, "declaracion.situacionPatrimonial.bienesInmuebles.bienInmueble")
    nivel = str(extraer_valor(doc, "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivel") or "").upper()

    if not bienes:
        return "SIN_DATO"
    if not nivel:
        return "SIN_DATO"

    n_declarante_compra = 0
    for b in bienes:
        titular = str(b.get("titular", "")).upper()
        forma = str(b.get("formaAdquisicion", "")).upper()

        if titular in ["DECLARANTE", "DECLARANTE Y PAREJA", "DECLARANTE Y TERCERO"]:
            if any(k in forma for k in ["COMPRA", "COMPRA-VENTA", "COMPRAVENTA"]):
                n_declarante_compra += 1

    if nivel not in UMBRAL_POR_NIVEL:
        return "SIN_DATO"

    q1, q3 = UMBRAL_POR_NIVEL[nivel]
    if n_declarante_compra < q1 or n_declarante_compra > q3:
        return "NO_CUMPLE"
    return "CUMPLE"

def procesar_metrica():
    client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
    db = client[DB_NAME]
    src = db[SOURCE_COLLECTION_NAME]
    tgt = db[METRICS_COLLECTION_NAME]

    total_docs = src.estimated_document_count()
    print(f"🚀 Procesando {METRIC_ID} con {total_docs:,} documentos...\n")

    skip = 0
    procesados = 0
    lote = 0

    try:
        while True:
            cursor = list(src.find({}, {
                "_id": 1,
                "declaracion.situacionPatrimonial.bienesInmuebles.bienInmueble": 1,
                "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivel": 1
            }).skip(skip).limit(BATCH_SIZE))

            if not cursor:
                break

            operaciones = []
            for doc in cursor:
                try:
                    resultado = evaluar_metrica(doc)
                except Exception as e:
                    resultado = "SIN_DATO"
                    print(f"[Error doc {doc.get('_id')}] {e}")

                operaciones.append(UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                ))

            if operaciones:
                tgt.bulk_write(operaciones)
                procesados += len(operaciones)
                lote += 1
                print(f"  🔄 Lote {lote:,} → {procesados:,}/{total_docs:,} procesados", flush=True)

            skip += BATCH_SIZE

        print("\n✅ Procesamiento completado.")
        print(f"Total final: {procesados:,}/{total_docs:,} documentos actualizados.")

    except Exception:
        traceback.print_exc()
    finally:
        client.close()
        print("🔒 Conexión cerrada.\n")

if __name__ == "__main__":
    procesar_metrica()


🚀 Procesando 3_4_02_BIENES_COMPRAVENTA_DECLARANTE con 8,942,592 documentos...

  🔄 Lote 1 → 10,000/8,942,592 procesados
  🔄 Lote 2 → 20,000/8,942,592 procesados
  🔄 Lote 3 → 30,000/8,942,592 procesados
  🔄 Lote 4 → 40,000/8,942,592 procesados
  🔄 Lote 5 → 50,000/8,942,592 procesados
  🔄 Lote 6 → 60,000/8,942,592 procesados
  🔄 Lote 7 → 70,000/8,942,592 procesados
  🔄 Lote 8 → 80,000/8,942,592 procesados
  🔄 Lote 9 → 90,000/8,942,592 procesados
  🔄 Lote 10 → 100,000/8,942,592 procesados
  🔄 Lote 11 → 110,000/8,942,592 procesados
  🔄 Lote 12 → 120,000/8,942,592 procesados
  🔄 Lote 13 → 130,000/8,942,592 procesados
🔒 Conexión cerrada.



KeyboardInterrupt: 